Prepare Training Data:

Obtain an existing text corpus of arbitrary size.

Determine a maximum sequence length (LM) for your distilled model. This will determine the length of the subsequences you'll extract from the text.

For each sequence length from 1 to LM, extract all possible subsequences from the text. These will serve as your training examples.

Generate Logits and Features:

Process each subsequence through the LLaMA2 70b model.

At each layer of the model, capture the logits and any other desired features (such as hidden states, attention scores, etc.). These features will provide the distilled knowledge.

Build Distilled Dataset:

For each subsequence, you'll have a set of logits and features from the LLaMA2 70b model. This forms your distilled dataset.

You can also include the ground truth tokens corresponding to each subsequence in the dataset. These tokens will serve as targets during training.

Train LLaMA2 7b with Distillation:

Train the LLaMA2 7b model using the distilled dataset.

During training, use a loss function that incorporates both the standard language modeling loss (using ground truth tokens) and a distillation loss. The distillation loss can be designed to match the logits and features from the LLaMA2 70b model.

The distillation loss encourages the LLaMA2 7b model to replicate the behavior of the larger LLaMA2 70b model.

Evaluate and Fine-Tune:

Evaluate the performance of the distilled LLaMA2 7b model on a validation set.

Fine-tune the training process as needed based on the evaluation results. You can adjust hyperparameters, modify the distillation loss, etc.

Test and Iterate:

Test the distilled model on various downstream tasks or benchmarks to see if the distilled knowledge from LLaMA2 70b has indeed improved the performance of LLaMA2 7b. Keep in mind that the exact implementation details, including the choice of distillation loss, optimization techniques, and model architecture, will depend on the specifics of the LLaMA2 models and your training framework. Additionally, this process might require significant computational resources, so be prepared for that as well.

Does it stand to reason that the model would be able to improve more easily training on these features instead of only the output token (top_k=1)?

Yes, training on additional features from the teacher model, such as logits and hidden states, in addition to the output tokens (top_k=1), can potentially lead to improved learning and better performance during distillation. There are a few reasons why including these features might be beneficial:

Richer Knowledge Transfer: Logits and hidden states contain more information about the model's internal representations and decision-making process than just the output tokens. By providing the student model with access to this richer information, it can potentially learn to capture more nuanced patterns and relationships in the data.

Regularization: Training with features from the teacher model can act as a form of regularization. The teacher's logits and hidden states can guide the student's learning process, helping to prevent overfitting and improving generalization to new data.

Robustness to Noise: When training solely on output tokens (top_k=1), the student model might be more susceptible to noisy or incorrect teacher predictions. Incorporating logits and features can help mitigate the impact of occasional teacher errors.

Model Depth and Capacity: If the teacher model (LLaMA2 70b) is significantly larger and more complex than the student model (LLaMA2 7b), training only on output tokens might not fully leverage the teacher's capacity. Including features from multiple layers allows the student model to benefit from the teacher's depth and expressiveness.

Faster Convergence: The student model might converge faster and require fewer iterations to achieve a certain level of performance when trained with richer features. This can lead to more efficient training.

